In [2]:
import pickle
import numpy as np
L=1e-3
Nx=2048
x = np.linspace(0.0, L, Nx)
with open("bte_1d.pkl", "rb") as f:
    ts, Tsnaps = pickle.load(f)
if True:#try:
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    mpl.rcParams.update({"figure.figsize": (7, 4)})
    for Ts in Tsnaps:#[np.array([1,6,-1])]:
        plt.plot(x * 1e6, Ts.T)#, label="?(x, t_final)")
    plt.xlabel("x [m]")
    #plt.ylabel("temperature perturbation ? [K]")
    plt.title("1D Inventory from Upwind BTE")
    #plt.legend()
    plt.tight_layout()
    plt.xlim(0,1000)
    plt.ylim(0,1e8)
    plt.savefig('bte_temps.png')
    plt.show()



OSError: [Errno 116] Stale file handle: 'bte_1d.pkl'

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
L=1e-3
Nx=2048
x = np.linspace(0.0, L, Nx)
with open("bte_1d.pkl", "rb") as f:
    ts, Tsnaps = pickle.load(f)
if False:#try:
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    mpl.rcParams.update({"figure.figsize": (7, 4)})
    for Ts in Tsnaps:#[np.array([1,6,-1])]:
        plt.plot(x * 1e6, Ts.T)#, label="?(x, t_final)")
    plt.xlabel("x [m]")
    #plt.ylabel("temperature perturbation ? [K]")
    plt.title("1D Inventory from Upwind BTE")
    #plt.legend()
    plt.tight_layout()
    plt.xlim(0,1000)
    plt.ylim(0,1e8)
    plt.savefig('bte_temps.png')
    plt.show()



In [ ]:
from numpy.linalg import svd,eig
fig, axs = plt.subplots(figsize=(6,4))
def decompose_field(x,T):
    from numpy.polynomial.legendre import legvander,legval
    # Rescale coordinates to [-1, 1]
    x_scaled = 2 * (x - x.min()) / (x.max() - x.min()) - 1
    
    # Get Vandermonde matrix (evaluated basis)
    deg_x = 10
    V = legvander(x_scaled, deg_x)  # shape: (N_points)
    
    # Project via inner product (or least-squares)
    # Result is vector of length (deg_x + 1)*(deg_y + 1)
    # Project T_flat onto the Legendre basis
    G = V.T @ V           # Gram matrix
    #print(V.shape,T.shape)
    b = V.T @ T           # Unnormalized projection
    c = np.linalg.solve(G, b)  # Solve for coefficients
    return c
def interp_temp(x,c):
    from numpy.polynomial.legendre import legval
    return legval(x, c,tensor=False) #np.sin(np.pi * x / Lx)  # W/m²
scale = lambda x: 2 * (x - x.min()) / (x.max() - x.min()) - 1
G = []
time_slice = np.linspace(1,13646,20,dtype=int)
ts_short = ts[time_slice]
Tshort = Tsnaps[time_slice]
for Ts in Tshort:
    c=decompose_field(x[100:1848],Ts.T[100:1848]*1e-9)
    G.append(c[:,0])
    T = interp_temp(scale(x[100:1848]),c)
    if True:
        plt.plot(x[100:1848] * 1e6,T)
        plt.plot(x[100:1848] * 1e6,Ts.T[100:1848]*1e-9,'o',markevery=100)
if True:
    plt.show()
#print(np.array(G))
fig, axs = plt.subplots(9,5,figsize=(8,12))
Gt = []
print(len(G))
cmp=11
for i in range(len(G)-11):
    Gt.append(G[i:11+i])
    X = np.array(G[i:11+i])
    Y = np.array(G[i+1:12+i])
    #print(X.shape,Y.shape)
    u,s,v = svd(X)
    #print(s)
    print(Y.shape,v.T[:,0:cmp].shape,u.T[0:cmp,:].shape)
    sinv = 1./s
    A = Y@v.T@np.diag(sinv)@u.T
    Ac = Y@v.T[:,0:cmp]@np.diag(sinv[0:cmp])@u.T[0:cmp,:]
    Lam,Xi = eig(A)
    Lam = np.diag(np.abs(Lam))
    #for ax,M in zip(axs[i,:],[u[:,0:cmp],np.diag(s[0:cmp]),v[0:cmp,:],A,Ac]):#,Lam,Xi.real,Xi.imag]):#[0,:]):
    for ax,M in zip(axs[i,:],[A,Ac,Lam,Xi.real,Xi.imag]):#[0,:]):
        im = ax.imshow(M, cmap='hot', vmin=-.2,vmax=.2,
               origin='upper', interpolation='nearest', aspect='equal')
if False:
    plt.xlim(0,1000)
    plt.ylim(0,1e-1)
    plt.show()
#fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
#plt.title(f'Green\'s Function after {Nt} steps')
#plt.xlabel('x (m)')
#plt.ylabel('y (m)')
plt.tight_layout()
#plt.savefig('matrix_evolution.png')
plt.show()



In [ ]:
G = []
time_slice = np.linspace(0,13648,13647,dtype=int)
#ts_short = ts[time_slice]
Tshort = Tsnaps#[time_slice]
for Ts in Tshort:
    c=decompose_field(x[100:1848],Ts.T[100:1848]*1e-9)
    G.append(c[:,0])
    T = interp_temp(scale(x[100:1848]),c)
    if False:
        plt.plot(x[100:1848] * 1e6,T)
        plt.plot(x[100:1848] * 1e6,Ts.T[100:1848]*1e-9,'o',markevery=100)
if False:
    plt.show()


In [ ]:
#print(np.array(G))
nA = 6
fig, axs = plt.subplots(nA,5,figsize=(10,4))
Gt = []
lAc = []
print(len(G))
cmp=4
#for i in range(0,len(G)-11,1):
strt = 1
major_stride = 2000
minor_stride = 50
npts = 6
for i,ax in enumerate(axs):
    #Gt.append(G[i:11+i])
    X = np.array(G[strt+i*major_stride:strt+i*major_stride+npts*minor_stride:minor_stride]).T
    Y = np.array(G[strt+i*major_stride+1:strt+i*major_stride+npts*minor_stride+1:minor_stride]).T
    print(X.shape,Y.shape)
    u,s,v = svd(X,full_matrices=False)
    #print(s)
    print(u.shape,v.shape)
    #print(Y.shape,v.T[:,0:cmp].shape,u.T[0:cmp,:].shape)
    sinv = 1./s
    A = Y@v.T
    #print(A.shape)
    A = Y@v.T@np.diag(sinv)@u.T
    Ac = Y@v.T[:,0:cmp]@np.diag(sinv[0:cmp])@u.T[0:cmp,:]
    lAc.append(A)
    for axi,M in zip(ax,[u[:,0:cmp],np.diag(s[0:cmp]),v[0:cmp,:],Ac,A]):
        im = axi.imshow(M, cmap='hot', vmin=-2,vmax=2,
           origin='upper', interpolation='nearest', aspect='equal')
#fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.show()
fig, axs = plt.subplots(nA,1,figsize=(8,12))

print(len(lAc))
for ax,i in zip(axs, list(range(0,nA))):#len(lAc),100))[:9]):
    im = ax.imshow(lAc[i], cmap='hot', #vmin=-50,vmax=50,
           origin='upper', interpolation='nearest', aspect='equal')
plt.tight_layout()
#plt.savefig('matrix_evolution.png')
plt.show()



In [ ]:
fig, axs = plt.subplots(figsize=(6,4))
nA = 100
lAc = []
cmp=5
#for i in range(0,len(G)-11,1):
strt = 1
major_stride = 5
minor_stride = 2
npts = 6
for i in range(nA):
    #Gt.append(G[i:11+i])
    X = np.array(G[strt+i*major_stride:strt+i*major_stride+npts*minor_stride:minor_stride]).T
    Y  = np.array(G[strt+i*major_stride+1:strt+i*major_stride+npts*minor_stride+1:minor_stride]).T
    u,s,v = svd(X,full_matrices=False)
    sinv = 1./s
    A = Y@v.T@np.diag(sinv)@u.T
    Ac = Y@v.T[:,0:cmp]@np.diag(sinv[0:cmp])@u.T[0:cmp,:]
    lAc.append(A)

Tstart=Tshort[0].T[100:1848]
c=decompose_field(x[100:1848],Tstart*1e-9)
print(c)
Tappx = interp_temp(scale(x[100:1848]),c)
print(Tappx)
#plt.plot(x[100:1848] * 1e6,Tappx)
#print(c)

for i, Ts in enumerate(Tshort[0:nA*major_stride:major_stride]):#[0:1]:
    
    Tappx = interp_temp(scale(x[100:1848]),c)
    if i%10==0:
        plt.plot(x[100:1848] * 1e6,Tappx)
        plt.plot(x[100:1848] * 1e6,Ts.T[100:1848]*1e-9,'o',markevery=100)

    c=Ac@c
    c[0]=decompose_field(x[100:1848],Ts.T[100:1848]*1e-9)[0]
    if False and i%10==0:
        print(c)
    #print(Ac)
    #print(c)
plt.ylim(0.09,0.11)
plt.show()



In [ ]:
fig, axs = plt.subplots(1,5,figsize=(12,4))
#fig, ax = plt.subplots(figsize=(12,4))
for j,ax in enumerate(axs):#[0,:]):
    im = ax.imshow(Gt[j], cmap='tab20', vmin=-.2,vmax=.2,
           origin='upper', interpolation='nearest', aspect='equal')
'''for j,ax in enumerate(axs[1,:]):
    im = ax.imshow(Gt[j+5], cmap='tab20', vmin=-5.,vmax=2.,
               origin='upper', interpolation='nearest', aspect='equal')'''
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
#plt.title(f'Green\'s Function after {Nt} steps')
#plt.xlabel('x (m)')
#plt.ylabel('y (m)')
plt.tight_layout()
#plt.savefig('matrix_evolution.png')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

def create_gif_from_matrices(matrices, output_gif, fps=30, figsize=(6,6)):
    """
    Create an animated GIF from a list of 2D NumPy matrices.

    Parameters:
    ----------
    matrices : list of np.ndarray
        List of 2D arrays to animate.
    output_gif : str
        Path to output GIF file.
    fps : int
        Frames per second in the GIF.
    figsize : tuple
        Size of the figure.
    """
    # Temporary directory to store the images
    temp_dir = "temp_images"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Create frames from the matrices
    for i, matrix in enumerate(matrices):
        fig, ax = plt.subplots(figsize=figsize)
        im = ax.imshow(matrix, cmap='viridis', interpolation='nearest')
        ax.set_xticks([])
        ax.set_yticks([])
        plt.close(fig)

        # Save each frame as an image
        frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
        fig.savefig(frame_path, dpi=150)
        plt.close(fig)
    
    # Create GIF using imageio
    with imageio.get_writer(output_gif, mode='I', duration=1/fps) as writer:
        for i in range(len(matrices)):
            frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
            image = imageio.imread(frame_path)
            writer.append_data(image)

    # Clean up temporary images
    for i in range(len(matrices)):
        os.remove(os.path.join(temp_dir, f"frame_{i:04d}.png"))
    os.rmdir(temp_dir)

    print(f"GIF saved to {output_gif}")

#create_gif_from_matrices(Gt, "animation.gif", fps=20)

